This example notebook shows how we can train a Factorization Machines classifier.
Factorization Machines ([FMs](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)) are a general predictor, that are able to perform well in problems with high sparsity, like recommender systems.
We employ TileDB as a storage engine for our training data and labels.
We will use the MovieLens 100K public data set, available [here](https://grouplens.org/datasets/movielens/100k/). We will first download the
MovieLens, which contains 100.000 ratings, by 943 users on 1682 items.
Continuing, we will use our TileDB support for PyTorch Sparse Dataloader API in order to train an FM classifier.
First, let's import what we need and download our data.

In [1]:
import os
import urllib.request
import numpy as np
import tiledb
import torch
import pandas as pd

## Dataset
Download MovieLens dataset.

In [2]:
url = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.data'
filename = 'u.data'
urllib.request.urlretrieve(url, filename)

('u.data', <http.client.HTTPMessage at 0x14b411eb0>)

Use pandas to display dataset in readable form

In [3]:
data = pd.read_csv(filename, sep="\t", header=None, engine='python')
data.columns = ["user_id", "item_id", "rating", "timestamp"]
data.drop(["timestamp"], axis=1, inplace=True)
display(data.head())
display(data.shape)

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


(100000, 3)

## Data Analysis / Sparsity
Before we apply the one-hot transformation let’s check the memory usage of our original data frame.

In [4]:
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3)
    print("Memory usage is " + str(mem) + " MB")

print_memory_usage_of_data_frame(data)

Memory usage is 2.4 MB


## Data transformation

Now, let’s apply the transformation and check the memory usage of the transformed data frame.

In [5]:
data_one_hot = pd.get_dummies(data, columns=['user_id', 'item_id'])
display(data_one_hot.head())
display(data_one_hot.shape)
print_memory_usage_of_data_frame(data_one_hot)

,rating,user_id_1,user_id_2,user_id_3,user_id_4,user_id_5,user_id_6,user_id_7,user_id_8,user_id_9,...,item_id_1673,item_id_1674,item_id_1675,item_id_1676,item_id_1677,item_id_1678,item_id_1679,item_id_1680,item_id_1681,item_id_1682
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(100000, 2626)

Memory usage is 263.3 MB


We will slice the dataset `user_movie` will be our `x_train` data transformed with one-hot encoding.
So we expect its schema to be the number of ratings as rows and binary columns for users + binary columns
for each item (in our case movies). This will lead to (100000, 2625)

The target data will be the `ratings`, which will include the ratings and thus will have a shape of (100000,1)

In [6]:
user_movie = data_one_hot[data_one_hot.columns.difference(['rating'])]
ratings = data['rating']



## Data Ingestion
Then we proceed with ingesting `user_movies` into sparse TileDB arrays as our training data and `rating` into dense TileDB
array as our target data. Here, we should point out that besides the
flexibility of TileDB in defining a schema, i.e., multiple dimensions, multiple attributes, compression etc,
we choose to define a simple schema. So, for a numpy array of D number of dimensions we create a dense TileDB array,
with the same number of dimensions, and a single attribute of data type numpy float32.

In [7]:
def get_schema(data: np.array, batch_size: int, sparse: bool) -> tiledb.ArraySchema:
    dims = [
        tiledb.Dim(
            name="dim_" + str(dim),
            domain=(0, data.shape[dim] - 1),
            tile=data.shape[dim] if dim > 0 else batch_size,
            dtype=np.int32,
        )
        for dim in range(data.ndim)
    ]

    # TileDB schema
    schema = tiledb.ArraySchema(
        domain=tiledb.Domain(*dims),
        sparse=sparse,
        attrs=[tiledb.Attr(name="features", dtype=np.float32)],
    )

    return schema

# Let's define an ingestion function
def ingest_in_tiledb(data: np.array, batch_size: int, uri: str, sparse: bool):
    schema = get_schema(data, batch_size, sparse)

    # Create the (empty) array on disk.
    tiledb.Array.create(uri, schema)

    # Ingest
    with tiledb.open(uri, "w") as tiledb_array:
        idx = np.nonzero(data) if sparse else slice(None)
        tiledb_array[idx] = {"features": data[idx]}

We transform our sliced/split data to numpy arrays, so we can ingest them in TileDB arrays

In [8]:
user_movie = user_movie.to_numpy()
ratings = ratings.to_numpy()

We ingest `user_movie` as sparse TileDB array and `ratings` as dense TileDB array

In [9]:
#Ingest images
ingest_in_tiledb(data=user_movie, batch_size=64, uri='training_images', sparse=True)

# Ingest labels
ingest_in_tiledb(data=ratings, batch_size=64, uri='training_labels', sparse=False)


We can now explore our TileDB arrays and check their structure.

In [10]:
user_movie_array = tiledb.open('training_images')
ratings_array = tiledb.open('training_labels')

print(user_movie_array.schema)
print(ratings_array.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='dim_0', domain=(0, 99999), tile=64, dtype='int32'),
    Dim(name='dim_1', domain=(0, 2624), tile=2625, dtype='int32'),
  ]),
  attrs=[
    Attr(name='features', dtype='float32', var=False, nullable=False),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=False,
  coords_filters=FilterList([ZstdFilter(level=-1)]),
)

ArraySchema(
  domain=Domain(*[
    Dim(name='dim_0', domain=(0, 99999), tile=64, dtype='int32'),
  ]),
  attrs=[
    Attr(name='features', dtype='float32', var=False, nullable=False),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=False,
  coords_filters=FilterList([ZstdFilter(level=-1)]),
)



## Model training
Although we used Factorization Machines as a reference model to create
our training set, here we will train a simple Logistic Regression model in Pytorch only
for demonstration purposes. Anyone can easily build any Model to train on the data.

### Declare Model Class

In [11]:
import torch.nn as nn
import torch.optim as optim

class LogisticRegression(nn.Module):
    def __init__(self, shape):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(shape[0], shape[1])

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

### Train the Model


In [12]:
from tiledb.ml.data_apis.pytorch_sparse import PyTorchTileDBSparseDataset

with tiledb.open('training_images') as x, tiledb.open('training_labels') as y:
    tiledb_dataset = PyTorchTileDBSparseDataset(x_array=x, y_array=y, batch_size=32)
    train_loader = torch.utils.data.DataLoader(tiledb_dataset, batch_size=None)

    #Number of ratings x (user + movies)
    datashape_x = (100000, 2625)

    logre = LogisticRegression(shape=(2625, 1))
    criterion = nn.MSELoss()
    optimizer = optim.SGD(logre.parameters(), lr=0.01, momentum=0.5)

    for epoch in range(1, 5):
        logre.train()
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = logre(inputs.to(torch.float))
            loss = criterion(outputs, labels.type(torch.FloatTensor))
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} Batch: {} Loss: {:.6f}'.format(
                epoch, batch_idx, loss.item()))

Train Epoch: 1 Batch: 0 Loss: 11.897947


/Users/konstantinostsitsimpikos/tileroot/TileDB-ML/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 1 Batch: 100 Loss: 1.434249
Train Epoch: 1 Batch: 200 Loss: 1.122545
Train Epoch: 1 Batch: 300 Loss: 1.596507
Train Epoch: 1 Batch: 400 Loss: 1.251316
Train Epoch: 1 Batch: 500 Loss: 1.563675
Train Epoch: 1 Batch: 600 Loss: 1.439507
Train Epoch: 1 Batch: 700 Loss: 1.296800
Train Epoch: 1 Batch: 800 Loss: 1.036165
Train Epoch: 1 Batch: 900 Loss: 1.375567
Train Epoch: 1 Batch: 1000 Loss: 0.937286
Train Epoch: 1 Batch: 1100 Loss: 1.061382
Train Epoch: 1 Batch: 1200 Loss: 1.314131
Train Epoch: 1 Batch: 1300 Loss: 1.419514
Train Epoch: 1 Batch: 1400 Loss: 1.049100
Train Epoch: 1 Batch: 1500 Loss: 1.067766
Train Epoch: 1 Batch: 1600 Loss: 1.188462
Train Epoch: 1 Batch: 1700 Loss: 0.808228
Train Epoch: 1 Batch: 1800 Loss: 1.374337
Train Epoch: 1 Batch: 1900 Loss: 1.305547
Train Epoch: 1 Batch: 2000 Loss: 1.811797
Train Epoch: 1 Batch: 2100 Loss: 1.568941
Train Epoch: 1 Batch: 2200 Loss: 1.187829
Train Epoch: 1 Batch: 2300 Loss: 1.315253
Train Epoch: 1 Batch: 2400 Loss: 1.187909
T

## Garbage Collection
We delete the RAW dataset `u.data`

In [13]:
os.remove(filename)